<a href="https://colab.research.google.com/github/shaoyinguo-portfolio/CorpGenie-exp/blob/main/Text2RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-core langchain-community langchain-openai langchain-text-splitters faiss-cpu pypdf langchain-huggingface --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 

In [11]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

from google.colab import drive, userdata
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

In [4]:
try:
    drive.mount('/content/drive')
    data_path = Path('/content/drive/MyDrive/Colab Notebooks/data')
    print('Mounted Google Drive')
except:
    data_path = Path('./data')
    print('Mounted local drive')

if not data_path.exists():
    data_path.mkdir()

TEXT_PATH = f'{data_path}/output_round2_10.txt'

KEYFRAME_PATH = f'{data_path}/key_frames'

Mounted at /content/drive
Mounted Google Drive


In [5]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

In [6]:
loader = TextLoader(TEXT_PATH)
docs = loader.load()
docs

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/output_round2_10.txt'}, page_content='```latex\nThe presentation focuses on TSMC and Intel\'s packaging process technologies, specifically CoWoS, EMIB, Foveros, and chiplets [ImageName: 1.00]. TSMC is recognized as a leader in semiconductor manufacturing, with strengths in both high yield of advanced nanometer nodes and in packaging technology. Among TSMC, Samsung, and Intel, TSMC\'s packaging process, including CoWoS and 3D fabrics, stands out. The presentation will primarily focus on CoWoS.\n\nThe Key Performance Indicators (KPIs) for packaging technology, often referred to as P3C2, include performance, power, packaging profile, cycle time, and cost [ImageName: 90.00, 1882.00]. Performance encompasses bandwidth (BW), Fmax, and functionality ("Hi, Function"). Power considers efficiency and thermal properties (Tj). The packaging profile involves footprint and thickness. Cycle time refers to the speed to market, 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/output_round2_10.txt'}, page_content="```latex\nThe presentation focuses on TSMC and Intel's packaging process technologies, specifically CoWoS, EMIB, Foveros, and chiplets [ImageName: 1.00]. TSMC is recognized as a leader in semiconductor manufacturing, with strengths in both high yield of advanced nanometer nodes and in packaging technology. Among TSMC, Samsung, and Intel, TSMC's packaging process, including CoWoS and 3D fabrics, stands out. The presentation will primarily focus on CoWoS."),
 Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/output_round2_10.txt'}, page_content='The Key Performance Indicators (KPIs) for packaging technology, often referred to as P3C2, include performance, power, packaging profile, cycle time, and cost [ImageName: 90.00, 1882.00]. Performance encompasses bandwidth (BW), Fmax, and functionality ("Hi, Function"). Power considers efficiency and thermal prop

In [12]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
vector_store.add_documents(splits)

['7f9e877a-cea6-456d-9502-0b3adcd68a3f',
 '3b515f02-a91b-4d22-b9d4-ca4c57ef1f4c',
 'a830822b-f650-4337-b117-6c621eeb82fe',
 'a6775bf0-8b6f-42f8-bb7f-e471741f4041',
 'c56edf02-85c7-43f7-a54f-10107d5db3a6',
 'cf6a74e1-918e-4b01-83bf-40f05d067617',
 'a44d8532-a71d-43d5-8142-7d59c507d418',
 '9b7dd806-9362-483c-80fa-9738af6afe54',
 '4bb1ccf4-ef61-4a32-9937-6b9401a5e419',
 '93df027c-28e0-450a-9afc-8a9428caae2f',
 '3d48c008-41ff-49fe-9e20-beef58fa131f',
 '5a744237-e117-4be8-83c6-a17733765568',
 '37f259f8-84ca-4df2-bf1f-671bb69f9fd3',
 '8c1d1016-3817-470e-8d57-036e0b3230e8',
 'aaa8fc76-6a6b-404e-9e4f-019fbf659fe8',
 'a35b5ae9-0c2b-48d8-babd-6bef13ae62c0',
 'f0c62c97-a65a-4e82-a709-5e8b006e7b55',
 '34bfa99a-278a-4e33-8971-e268acbcfd81',
 'c3c2d899-ba72-4c5e-ac1e-a13bf2b4c4f1',
 'f9ddcb53-f1ec-4573-aa51-216cf5266e4a',
 'd7912b63-a6ed-4521-a8ce-7941e961fa14',
 'fc617014-9c4a-4804-81f5-6648b74e7498',
 '1a0c99bb-54d5-41cf-b236-f1a3e46ec5fe',
 'f066197b-7466-4ba6-b563-016e2a7bb16c',
 '43ecaff6-9dac-

In [14]:
# save vectorstore
vector_store.save_local(data_path)

In [15]:
# load
loaded_vectorstore = FAISS.load_local(
    data_path,
    embeddings,
    allow_dangerous_deserialization=True
)

In [23]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [24]:
from langchain.agents import create_agent

model = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
    model="google/gemini-2.0-flash-001" # "google/gemini-2.0-flash-exp:free"
)

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a tech document. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [28]:
query = (
    "Why TSMC CoWoS is the most popular packaging technology as oppose to Intel EMIB and Foveros?\n\n"
    "Once you get the answer, write an organized, detailed and logical paragraph."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Why TSMC CoWoS is the most popular packaging technology as oppose to Intel EMIB and Foveros?

Once you get the answer, write an organized, detailed and logical paragraph.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (tool_0_retrieve_context_uZ0hZ8pDE2Wdnh8RmIk2)
 Call ID: tool_0_retrieve_context_uZ0hZ8pDE2Wdnh8RmIk2
  Args:
    query: TSMC CoWoS vs Intel EMIB and Foveros popularity
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': '/content/drive/MyDrive/Colab Notebooks/data/output_round2_10.txt'}
Content: The presenter states that the limitation of the interposer size for CoWoS is no longer valid with TSMC's advancements in reticle sizes. Additionally, despite CoWoS's challenges with stress, EMIB faces thermal problems due to the use of polymers. The presenter concl

TSMC's CoWoS (Chip-on-Wafer-on-Substrate) packaging technology has gained popularity over Intel's EMIB (Embedded Multi-die Interconnect Bridge) and Foveros for several reasons. CoWoS benefits from dense, fine-pitch interconnects and CTE (Coefficient of Thermal Expansion) matching, which reduces stress on the die's backend and the likelihood of failures. Concerns about interposer size limitations are becoming less valid due to TSMC's advancements in reticle sizes, with TSMC moving to solve that problem to make it so big with its 3x and 4x reticle sizes in 2023, moving to 6x reticle sizes soon. While EMIB leverages existing organic packaging technology and enables larger die counts, it can face thermal problems due to the use of polymers. Foveros involves die-to-die stacking with direct copper bonding, eliminating the need for TSVs, but its bump pitch is 36μm. TSMC's larger IP portfolio and customer-focused business model, compared to Intel's historically more internal-focused approach, have also contributed to CoWoS's wider adoption.
